In [ ]:
! pip install -r requirments.txt

In [ ]:
import librosa
from transformers import AutoFeatureExtractor, AutoModel
from transformers import WhisperFeatureExtractor, WhisperModel
import joblib
import numpy as np
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import os
from mutagen.mp3 import MP3
import torchaudio
import torch
from tqdm import tqdm
import opensmile

In [ ]:

audio_folder = "Path_to_audio_folder"
metadataPath = "Path_to_metadata_csv_file"

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define paths
dataType = 'inference'

original_sampling_rate = 48000  
target_sampling_rate = 16000    

def load_and_resample_audio(file_path, target_sampling_rate):
    waveform, original_rate = torchaudio.load(file_path)
    original_length = waveform.shape[-1] / original_rate
    
    if original_rate != target_sampling_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=original_rate, new_freq=target_sampling_rate)
        waveform = resampler(waveform)
    return waveform.squeeze(), original_length

# Generate model for embedding extraction using OpenSMILE
def audio_embeddings_model(model_name):
    if model_name == "compare":
        model = opensmile.Smile(
            feature_set=opensmile.FeatureSet.ComParE_2016,
            feature_level=opensmile.FeatureLevel.Functionals,
        )
    elif model_name == "egemaps":
        model = opensmile.Smile(
            feature_set=opensmile.FeatureSet.eGeMAPSv02,
            feature_level=opensmile.FeatureLevel.Functionals,
        )
    return model

# Embedding extraction function for OpenSMILE features
def audio_embeddings(audio_list, model, sampling_rate):
    embeddings_list = []
    for audio in tqdm(audio_list, desc="Extracting OpenSMILE features"):
        embeddings = model.process_signal(audio.numpy(), sampling_rate)
        embeddings_list.append(embeddings.values.flatten())
    return embeddings_list

# Generic function to extract embeddings from a Transformer-based audio model
def extract_embeddings_from_transformer_model(audio_list, model_name, sampling_rate):
    print(f"Extracting embeddings using {model_name}...")
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    embeddings_list = []
    for audio in tqdm(audio_list, desc=f"Extracting {model_name} embeddings"):
        inputs = feature_extractor(audio.numpy(), sampling_rate=sampling_rate, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = torch.mean(outputs.last_hidden_state, dim=1).cpu().numpy()
        embeddings_list.append(embeddings.squeeze())
    return embeddings_list

# Function to extract encoder embeddings from Whisper
def extract_whisper_embeddings(audio_list, model_name, sampling_rate):
    print(f"Extracting embeddings using {model_name} (Whisper) ...")
    feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
    model = WhisperModel.from_pretrained(model_name).to(device)
    model.eval()

    embeddings_list = []
    for audio in tqdm(audio_list, desc=f"Extracting {model_name} embeddings"):
        audio_np = audio.numpy()
        inputs = feature_extractor(audio_np, sampling_rate=sampling_rate, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = model.encoder(**inputs)
            embeddings = torch.mean(outputs.last_hidden_state, dim=1).cpu().numpy()
        embeddings_list.append(embeddings.squeeze())
    return embeddings_list

# Function to extract additional features
def extract_additional_features(audio, sampling_rate):
    audio = audio.numpy()
    mfccs = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sampling_rate)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sampling_rate)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sampling_rate, roll_percent=0.85)
    zero_crossings = librosa.feature.zero_crossing_rate(y=audio)
    pitches, magnitudes = librosa.core.piptrack(y=audio, sr=sampling_rate)
    pitch = np.max(pitches, axis=0)
    features = np.hstack([
        np.mean(mfccs, axis=1),
        np.mean(chroma, axis=1),
        np.mean(spectral_centroid, axis=1),
        np.mean(spectral_rolloff, axis=1),
        np.mean(zero_crossings, axis=1),
        np.mean(pitch)
    ])
    return features


# Load and process MP3 files
audio_files = [os.path.join(audio_folder, file) for file in os.listdir(audio_folder) if file.endswith(".mp3")]

data = [load_and_resample_audio(file, target_sampling_rate) for file in tqdm(audio_files)]
audio_list = [item[0] for item in data]
audio_lengths = [item[1] for item in data]
egemaps_model = audio_embeddings_model("egemaps")
print("Extracting eGeMAPS")
egemaps_features = audio_embeddings(audio_list, egemaps_model, target_sampling_rate)
wav2vec2_model_name = "facebook/wav2vec2-large-960h-lv60-self"
whisper_model_name = "openai/whisper-medium"
print("Extracting Wav2Vec2 ")
wav2vec2_features = extract_embeddings_from_transformer_model(audio_list, wav2vec2_model_name, target_sampling_rate)
print("Extracting Whisper")
whisper_features = extract_whisper_embeddings(audio_list, whisper_model_name, target_sampling_rate)

additional_features_list = []
for audio in tqdm(audio_list, desc="Extracting additional features"):
    additional_features = extract_additional_features(audio, target_sampling_rate)
    additional_features_list.append(additional_features)

additional_feature_columns = (
    [f"mfcc_{i}" for i in range(1, 14)]
    + [f"chroma_{i}" for i in range(1, 13)]
    + ["spectral_centroid", "spectral_rolloff", "zero_crossing_rate", "pitch"]
)
additional_features_df = pd.DataFrame(additional_features_list, columns=additional_feature_columns)

# Create DataFrame for all features
features_df = pd.DataFrame(egemaps_features, columns=egemaps_model.column_names)
features_df["Wav2Vec2_embeddings"] = list(wav2vec2_features)
features_df["Whisper_embeddings"] = list(whisper_features)
features_df["file_name"] = [os.path.basename(file) for file in audio_files]
features_df["augmentation_type"] = "original"  # tag as original
features_df = pd.concat([features_df, additional_features_df], axis=1)
embeddings_features_df = pd.DataFrame(features_df['Wav2Vec2_embeddings'].tolist(), columns=[f'Embedding1_{i+1}' for i in range(len(features_df['Wav2Vec2_embeddings'][0]))])
features_df_exctracted = pd.concat([features_df.drop(columns=['Wav2Vec2_embeddings']), embeddings_features_df], axis=1)
embeddings_features_df = pd.DataFrame(features_df_exctracted['Whisper_embeddings'].tolist(), columns=[f'Embedding2_{i+1}' for i in range(len(features_df_exctracted['Whisper_embeddings'][0]))])
features_df_exctracted = pd.concat([features_df_exctracted.drop(columns=['Whisper_embeddings']), embeddings_features_df], axis=1)
features_df_exctracted.to_csv("Features/"+dataType+"_audio_features.csv", index=False)

In [ ]:
model = joblib.load('Model/model.joblib')
TestData = pd.read_csv("Features/"+dataType+"_audio_features.csv")
TestData['uid'] = TestData['file_name'].apply(lambda x: x[:-4])
meta = pd.read_csv(metadataPath)[['uid','age','gender']]
meta['gender'] = meta['gender'].apply(lambda x : 1 if x == 'male' else 0)
test_data = pd.merge(TestData, meta, on='uid')
test_data.drop('augmentation_type', axis=1, inplace=True)

In [ ]:
uids = test_data['uid'].values
X_test = test_data.drop(columns=['uid', 'augmentation_type', 'file_name'], errors='ignore')
y_test_proba = model.predict_proba(X_test)
feature_names = X_test.columns.tolist()

In [ ]:
proba_df = pd.DataFrame(
    y_test_proba,
    columns=['diagnosis_control', 'diagnosis_mci', 'diagnosis_adrd']
)
proba_df['uid'] = uids
proba_df['confidence'] = proba_df[['diagnosis_control', 'diagnosis_mci', 'diagnosis_adrd']].max(axis=1)
final_proba_df = proba_df.loc[proba_df.groupby('uid')['confidence'].idxmax()]
final_proba_df = final_proba_df.drop(columns=['confidence'])
final_proba_df.reset_index(drop=True, inplace=True)

final_proba_df

In [ ]:
final_proba_df.to_csv("inference_results.csv")